In [51]:
!pip install openpyxl

                                              0.0/250.0 kB ? eta -:--:--
     ------------------                     122.9/250.0 kB 2.4 MB/s eta 0:00:01
     -------------------------------------- 250.0/250.0 kB 2.6 MB/s eta 0:00:00


In [2]:
import random
from time import sleep
from bs4 import BeautifulSoup

In [65]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
import pyautogui

In [58]:
import os
os.environ['PATH'] += ":/content/geckodriver"

In [110]:
def coletar_dados(cnpj, data):
  ops = Options()
  ops.add_argument("start-maximized")
  ops.add_argument('window-size=1920x1080')
  user_agent = 'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'
  ops.add_argument(f'--user-agent={user_agent}')

  browser = webdriver.Firefox(service=Service(), options=ops)
    
  pyautogui.PAUSE = 0.4
  #pyautogui.click(907,1052,2)
    
  pyautogui.moveTo(960+random.randint(-200,200),540+random.randint(-200,200),2)

  browser.get('https://grpfordam.sefin.fortaleza.ce.gov.br/grpfor/pagesPublic/iptu/damIptu/imprimirDamIptu.seam')
  sleep(2.5)

  pyautogui.moveTo(127,500+random.randint(-3,3),2, pyautogui.easeInOutQuad)
  pyautogui.click(127,500+random.randint(-3,3),1)

  pyautogui.moveTo(226,500+random.randint(-4,4),2+random.randint(-1,1), pyautogui.easeInOutQuad)
  pyautogui.click(226,500+random.randint(-4,4),1)

  pyautogui.write(cnpj)

  pyautogui.moveTo(382,500+random.randint(-4,4),2+random.randint(-1,1), pyautogui.easeInOutQuad)
  pyautogui.click(382,500+random.randint(-4,4),1)

  pyautogui.write(data)

  pyautogui.moveTo(530,500+random.randint(-5,5),2+random.randint(-1,1), pyautogui.easeInOutQuad)
  pyautogui.click(530,500+random.randint(-5,5),1)
  sleep(2.5)

  return browser

In [111]:
def coletarDadosPagina(cnpj, data, timeout):
  if timeout == 0:
    return None

  browser = coletar_dados(cnpj, data)
  result = browser.page_source

  site = BeautifulSoup(result, 'html.parser')

  alerta = site.find('dt', attrs={'class':'alert alert-danger'})
  if alerta:
    mensagem = alerta.find('span', attrs={'class':'rich-messages-label'})
    if "robô" in mensagem.text:
      sleep(2)
      browser.quit()
      coletarDadosPagina(cnpj, data, timeout - 1)
    else:
      return browser
  else:
    return browser

In [117]:
def coletarDadosImovel(cnpj, data, timeout):

  result = coletarDadosPagina(cnpj, data, timeout)
  if result == None:
    return [0, cnpj, data]

  browser = result
  
  lista_imoveis = []
  site = BeautifulSoup(browser.page_source, 'html.parser')

  dados_imovel_unico = site.find('span', attrs= {'id': 'pmfInclude:cadastroForm:dadosImovel'})
  tabela = site.find('table', attrs= {'id':'pmfInclude:cadastroForm:dataTable'})
  if tabela:
    alteracao = True
    while alteracao:
      imoveis = site.find_all('tr', attrs= {'class':'rich-table-row'})

      for imovel in imoveis:
        dados_imovel_tds = imovel.find_all('td')

        lista_imovel = []
        for dados_imovel_td in dados_imovel_tds[0:4]:
          lista_imovel.append(dados_imovel_td.text)

        lista_imoveis.append(lista_imovel)

      botao_bs = site.find_all('td', attrs = {'class':'rich-datascr-button'})
      if botao_bs:
        if botao_bs[3]['class'][0] != 'rich-datascr-button-dsbld':
          textoOnClick = "Event.fire(this, 'rich:datascroller:onscroll', {'page': 'next'});"
          botao = browser.find_element(By.XPATH, f'//td[@onclick="{textoOnClick}"]')
          botao.click()
          sleep(2)
          site = BeautifulSoup(browser.page_source, 'html.parser')
        else:
          alteracao = False
      else:
        alteracao = False
    sleep(2)
    browser.quit()
    return lista_imoveis
  elif dados_imovel_unico:
    dados_imovel_unico_fieldset = dados_imovel_unico.find('fieldset')
    dados_imovel_unico_tds = dados_imovel_unico_fieldset.find_all('td')
    lista_imovel_unico = [2]
    for dados_imovel_unico_td in dados_imovel_unico_tds[2::3]:
      lista_imovel_unico.append(dados_imovel_unico_td.text)
    
    sleep(2)
    browser.quit()
    return lista_imovel_unico
  else:
    mensagem_texto = ''
    alerta = site.find('dt', attrs={'class':'alert alert-danger'})
    if alerta:
      mensagem = alerta.find('span', attrs={'class':'rich-messages-label'})
      mensagem_texto = mensagem.text

    sleep(2)
    browser.quit()
    return [1, cnpj, data, mensagem_texto]

In [88]:
lista_imovel = coletarDadosImovel('12279725000140', '04111987', 1)
print(lista_imovel)

[['56387-0', 'R CANUTO DE AGUIAR 1077 200 MEIRELES CEP 60160-120', '19 - 68 - 46 - 2', '12.***.***/0001-40 - IMOBILIARIA ALVES ****** ****'], ['77313-1', 'R PEDRO PEREIRA 460 SALA 403 CENTRO CEP 60035-000', '1 - 13 - 56 - 41', '12.***.***/0001-40 - IMOBILIARIA ALVES ****** ****'], ['77325-5', 'R PEDRO PEREIRA 460 SALA 413 CENTRO CEP 60035-000', '1 - 13 - 56 - 51', '12.***.***/0001-40 - IMOBILIARIA ALVES ****** ****'], ['77333-6', 'R PEDRO PEREIRA 460 SALA 510 CENTRO CEP 60035-000', '1 - 13 - 56 - 63', '12.***.***/0001-40 - IMOBILIARIA ALVES ****** ****']]


In [52]:
import pandas as pd
df = pd.read_excel('lista_fortaleza.xlsx')

In [95]:
lista_nomes = ['Pessoa Jurídica', 'Data Abertura RFB', 'Inscrição', 'Cartografia', 'Endereço de Localização', 'Correspondência', 'Proprietário/Adquirente/Titular']
#lista_nomes = ['Pessoa Jurídica', 'Data Abertura RFB', 'Inscrição', 'Endereço', 'Cartografia', 'Titular']
lista_cnpjs = []
lista_data_abertura = []
lista_inscricao = []
lista_cartografia = []
lista_endereco = []
lista_correspondencia = []
lista_titular = []

cadastro_nao_encontrado = []

In [121]:
%%time
def loop_imoveis(index):
    lista_imoveis = coletarDadosImovel(df.loc[index, 'Pessoa Jurídica'], df.loc[index, 'Data Abertura RFB'], 2)
    if not isinstance(lista_imoveis[0], list):
      lista_imovel = lista_imoveis
      if lista_imovel[0] == 0:
        print(f'Index[{index}],Lista{lista_imovel[1:3]}, Erro no Captcha')
        sleep(120)
        loop_imoveis(index)
      elif lista_imovel[0] == 1:
        print(f'Index[{index}],Lista{lista_imovel[1:4]}')
        cadastro_nao_encontrado.append(lista_imovel[1:4])
      else:
        print(f'Index[{index}]')
        lista_cnpjs.append(df.loc[index, 'Pessoa Jurídica'])
        lista_data_abertura.append(df.loc[index, 'Data Abertura RFB'])
        lista_inscricao.append(lista_imovel[1])
        lista_cartografia.append(lista_imovel[2])
        lista_endereco.append(lista_imovel[3])
        lista_correspondencia.append(lista_imovel[4])
        lista_titular.append(lista_imovel[5])
    else:
      print(f'Index[{index}], Imóveis[{len(lista_imoveis)}]')
      for lista_imovel in lista_imoveis:
        lista_cnpjs.append(df.loc[index, 'Pessoa Jurídica'])
        lista_data_abertura.append(df.loc[index, 'Data Abertura RFB'])
        lista_inscricao.append(lista_imovel[0])
        lista_endereco.append(lista_imovel[1])
        lista_cartografia.append(lista_imovel[2])
        lista_titular.append(lista_imovel[3])
        lista_correspondencia.append('')

for index in range (7071,67413): #(0,67413)(67413,134825)//len -> 134824
        loop_imoveis(index)
  

Index[5972],Lista['06.169.942/0001-84', '24/03/2004', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[5973],Lista['72.469.950/0001-70', '21/07/1993', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[5974],Lista['35.363.786/0001-96', '31/10/2019', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[5975],Lista['12.734.904/0001-20', '23/10/2010', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[5976],Lista['10.318.292/0001-69', '01/09/2008', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[5977],Lista['03.975.023/0001-00', '01/08/2000', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[5978],Lista['14.407.741/0001-04', '04/10/2011', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[5979],Lista['07.384.905/0001-51', '16/05/2005', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[5980],Lista['41.436.601/0001-29', '04/12/1991', 'Não existe nenhum i

Index[6047],Lista['11.811.411/0001-83', '21/07/1987', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6048],Lista['72.442.338/0001-04', '13/07/1993', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6049],Lista['15.243.952/0001-03', '22/03/2012', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6050],Lista['06.314.410/0001-93', '03/06/2004', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6051],Lista['01.639.397/0001-29', '07/01/1997', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6052],Lista['13.782.892/0001-71', '01/06/2011', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6053],Lista['09.594.464/0001-48', '03/06/2008', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6054],Lista['03.794.652/0002-14', '17/01/2001', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6055],Lista['17.918.279/0001-90', '11/04/2013', 'Não existe nenhum im

Index[6122],Lista['24.597.592/0001-39', '14/04/2016', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6123],Lista['14.263.155/0001-25', '18/07/2011', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6124],Lista['23.843.621/0001-32', '15/12/2015', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6125],Lista['31.362.903/0001-46', '28/08/2018', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6126],Lista['09.517.707/0001-44', '11/09/1979', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6127],Lista['07.067.207/0001-22', '04/11/2004', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6128],Lista['11.230.918/0001-43', '16/10/2009', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6129],Lista['00.261.912/0001-17', '25/10/1994', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6130],Lista['63.470.298/0001-13', '29/05/1991', 'Documento não loca

Index[6195],Lista['24.729.988/0001-92', '05/05/2016', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6196],Lista['07.337.595/0001-14', '31/01/1984', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6197],Lista['03.149.649/0001-67', '13/05/1999', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6198],Lista['07.844.228/0001-07', '18/03/1974', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6199],Lista['13.201.716/0001-07', '03/02/2011', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6200],Lista['03.689.380/0001-01', '08/03/2000', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6201],Lista['00.150.238/0001-01', '18/08/1994', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6202],Lista['04.889.642/0001-44', '05/03/2001', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6203],Lista['13.901.949/0001-04', '29/06/2011', 'Não existe nenhum imóv

Index[6265],Lista['26.099.372/0001-00', '03/09/2016', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6266],Lista['72.131.238/0001-67', '19/04/1993', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6267],Lista['23.521.411/0001-28', '22/10/2015', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6268],Lista['07.338.544/0006-11', '08/09/1977', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6269],Lista['07.332.554/0001-35', '09/12/1983', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6270], Imóveis[24]
Index[6271],Lista['23.553.832/0001-30', '27/10/2015', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6272],Lista['15.013.208/0001-12', '07/02/2012', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6273],Lista['41.307.877/0001-07', '01/11/1991', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6274],Lista['22.947.625/0001-06', '29/07/2015',

...

Index[6850],Lista['22.386.793/0001-61', '06/05/2015', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6851],Lista['18.613.788/0001-77', '05/08/2013', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6852],Lista['03.265.129/0001-10', '29/12/1998', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6853],Lista['15.415.702/0001-03', '23/04/2012', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6854],Lista['02.102.111/0001-34', '16/09/1997', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6855],Lista['20.699.360/0001-30', '15/07/2014', 'Dados da pessoa divergentes, favor atualizar cadastro no Portal de Serviços do Contribuinte - eSEFIN , clique aqui!']
Index[6856],Lista['23.563.224/0001-07', '07/03/1989', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6857],Lista['40.900.117/0001-46', '18/02/2021', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6858],Lista[

Index[6921],Lista['02.954.372/0001-82', '30/09/1998', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6922],Lista['23.720.261/0001-81', '20/06/1989', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6923],Lista['04.208.439/0001-65', '22/12/2000', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6924],Lista['07.361.967/0001-48', '18/08/1989', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6925],Lista['20.870.870/0001-29', '28/07/2014', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6926],Lista['13.021.858/0001-84', '17/12/2010', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6927],Lista['63.378.731/0001-95', '28/02/1991', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6928],Lista['08.067.844/0001-61', '05/06/2006', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6929],Lista['19.762.261/0001-77', '20/02/2014', 'Documento não loca

Index[6994], Imóveis[4]
Index[6995],Lista['04.885.243/0001-05', '28/01/2002', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6996],Lista['17.113.195/0001-89', '05/11/2012', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[6997],Lista['26.379.673/0001-98', '19/10/2016', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6998],Lista['23.569.115/0001-05', '29/10/2015', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[6999],Lista['05.884.862/0001-48', '19/09/2003', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[7000]
Index[7001],Lista['97.448.443/0001-81', '20/04/1994', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[7002],Lista['27.747.186/0001-01', '16/05/2017', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[7003],Lista['21.539.426/0001-98', '09/12/2014', 'Documento não localizado no cadastro de pessoas da SEFIN.']
Index[7004],Lista['09.314.159/0001-55', '

Index[7067],Lista['28.804.403/0001-02', '05/10/2017', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[7068], Imóveis[6]
Index[7069],Lista['43.559.979/0001-45', '17/09/2021', 'Não existe nenhum imóvel vinculado ao documento informado.']
Index[7070]
Index[7071],Lista['07.691.683/0001-10', '14/11/2005', '']
Index[7072],Lista['41.391.780/0001-25', '20/02/1992', '']
Index[7073],Lista['07.886.419/0001-31', '15/02/1974', '']


FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.

In [120]:
preparacao_df_imoveis = list(zip(lista_cnpjs, lista_data_abertura, lista_inscricao, lista_cartografia,
                                 lista_endereco, lista_correspondencia, lista_titular))
df_imoveis = pd.DataFrame(preparacao_df_imoveis, columns = lista_nomes)
df_cadastro_nao_encontrado = pd.DataFrame(cadastro_nao_encontrado, columns = ['Pessoa Jurídica', 'Data Abertura RFB', 'Mensagem Erro'])

df_imoveis.to_excel('dados_imoveis_4.xlsx', index=False)
df_cadastro_nao_encontrado.to_excel('dados_cadastro_nao_encontrado_4.xlsx', index=False)

In [84]:
ops = Options()
ops.add_argument("start-maximized")
ops.add_argument('window-size=1920x1080')

cnpj = '12279725000140'
data = '04111987'

browser = webdriver.Firefox(service=Service(), options=ops)
clique = random.randint(-5,5)

#pyautogui.click(907,1052,2)

pyautogui.moveTo(960+random.randint(-200,200),540+random.randint(-200,200),2)

browser.get('https://grpfordam.sefin.fortaleza.ce.gov.br/grpfor/pagesPublic/iptu/damIptu/imprimirDamIptu.seam')
sleep(5)

pyautogui.moveTo(127,500+random.randint(-3,3),3, pyautogui.easeInOutQuad)
pyautogui.click(127,500+random.randint(-3,3),1)
sleep(1)
pyautogui.moveTo(226,500+random.randint(-4,4),2+random.randint(-1,1), pyautogui.easeInOutQuad)
pyautogui.click(226,500+random.randint(-4,4),1)
sleep(1)
pyautogui.write(cnpj)
sleep(1)
pyautogui.moveTo(382,500+random.randint(-4,4),2+random.randint(-1,1), pyautogui.easeInOutQuad)
pyautogui.click(382,500+random.randint(-4,4),1)
sleep(1)
pyautogui.write(data)
sleep(1)
pyautogui.moveTo(530,500+random.randint(-5,5),2+random.randint(-1,1), pyautogui.easeInOutQuad)
pyautogui.click(530,500+random.randint(-5,5),1)
sleep(5)



In [78]:
delay = random.randint(2,8)

  browser.get('https://grpfordam.sefin.fortaleza.ce.gov.br/grpfor/pagesPublic/iptu/damIptu/imprimirDamIptu.seam')
  sleep(4)

  pyautogui.size()
  pyautogui.position()

  radioButtom = browser.find_element(By.ID, 'pmfInclude:cadastroForm:tipoDecorate:j_id334:1')
  radioButtom.click()
  sleep(delay)

  elemento1 = browser.find_element(By.ID, 'pmfInclude:cadastroForm:cpfDec:cnpj')
  elemento1.click()
  elemento1.send_keys(cnpj)

  elemento2 = browser.find_element(By.ID, 'pmfInclude:cadastroForm:dataNascimentolDec:dataNascimentoInputDate')
  elemento2.click()
  elemento2.send_keys(data)

  buttomPesquisar = browser.find_element(By.ID, 'pmfInclude:cadastroForm:dataNascimentolDec:botaoRecuperarImovelNaoLogado') #para elementos
  sleep(6)
  buttomPesquisar.click()
  sleep(6)

In [104]:
browser.quit()

In [118]:
print(lista_cnpjs)

['07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.612/0001-68', '07.836.6